# Project Report
The project is open-ended with the exact goal to be decided amongst your group but the theme is around simulations and data relating to the coronavirus epidemic. You should make a program or several programs that can either run simulations or produce plots based on real data.

Along with your code there should be a group Project Report explaining your code and what it does and including some figures that are output from your code or perhaps links to videos or any other output.
